In [1]:
import torch
from torch import nn
from torch import optim

import torchvision
import torchvision.transforms as T

In [2]:
from data.util import get_dataset, IdxDataset

In [30]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=32
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=32, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
i_s = 3*28*28

model = AE(input_shape=i_s).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [32]:
train_dataset = get_dataset(
            "cmnist",
            data_dir="data/",
            dataset_split="train",
            transform_split="train",
            percent="5pct"
        )
#test_dataset = get_dataset(
#            "cmnist",
#            data_dir="data/",
#            dataset_split="train",
#            transform_split="train",
#            percent="5pct"
#        )

In [33]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

#train_dataset = torchvision.datasets.MNIST(
#    root="~/torch_datasets", train=True, transform=transform, download=True
#)

#test_dataset = torchvision.datasets.MNIST(
#    root="~/torch_datasets", train=False, transform=transform, download=True
#)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True
)

#test_loader = torch.utils.data.DataLoader(
#    test_dataset, batch_size=32, shuffle=False, num_workers=4
#)

In [9]:
train_loader

In [34]:
epochs = 20

In [35]:
for epoch in range(0,epochs):
    loss = 0
    for batch_features, attr, _ in train_loader:
        #print(idx)
        
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        
        batch_features = batch_features.view(-1, i_s).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/20, loss = 0.025624
epoch : 2/20, loss = 0.017730
epoch : 3/20, loss = 0.016594
epoch : 4/20, loss = 0.016093
epoch : 5/20, loss = 0.015769
epoch : 6/20, loss = 0.015507
epoch : 7/20, loss = 0.015329
epoch : 8/20, loss = 0.015162
epoch : 9/20, loss = 0.014998
epoch : 10/20, loss = 0.014848
epoch : 11/20, loss = 0.014725
epoch : 12/20, loss = 0.014610
epoch : 13/20, loss = 0.014461
epoch : 14/20, loss = 0.014328
epoch : 15/20, loss = 0.014242
epoch : 16/20, loss = 0.014157
epoch : 17/20, loss = 0.014093
epoch : 18/20, loss = 0.014018
epoch : 19/20, loss = 0.013951
epoch : 20/20, loss = 0.013894


In [ ]:
batch_features.view(-1,28,28).shape

In [25]:
idx = 3

In [36]:
for idx in range(10):

    i =  batch_features.view(-1,3,28,28)[idx]    

    transform = T.ToPILImage()
    img = transform(i)
    img.show()

    i =outputs.view(-1,3,28,28)[idx]    

    transform = T.ToPILImage()
    img = transform(i)
    img.show()



In [37]:
!gpustat

485571ad9a08                   Mon Sep 12 17:36:42 2022  470.141.03
[0] NVIDIA GeForce GTX 1080 Ti | 63'C,  14 % |  1512 / 11178 MB |
[1] NVIDIA GeForce GTX 1080 Ti | 28'C,   0 % |     8 / 11178 MB |
[2] NVIDIA GeForce GTX 1080 Ti | 31'C,   0 % |     8 / 11178 MB |


In [27]:
i =outputs.view(-1,3,28,28)[idx]    
i
transform = T.ToPILImage()
img = transform(i)
img.show()


